## Basic processing of the data file
- count entries by scientific name
- create new column that's just the genus
    - if SN only has one word then keep it, otherwise take the first word  
- count entries by genus

Summary: 
- some columns are entirely null, we will get rid of them at some point
- by scientific name, there are 364 different species 
- by genus name, there are 109 different genera. 13 of these have only one data point, 39 of these have less than 10

In [54]:
import pandas as pd

In [55]:
df = pd.read_csv('MLNS_Insects.csv')

In [63]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13461 entries, 0 to 13460
Data columns (total 49 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   cat_num                      13461 non-null  float64
 1   format                       13461 non-null  object 
 2   common_name                  11191 non-null  object 
 3   genus                        13461 non-null  object 
 4   scientific_name              13461 non-null  object 
 5   background_species           128 non-null    object 
 6   recordist                    13461 non-null  object 
 7   date                         13437 non-null  object 
 8   year                         13442 non-null  float64
 9   month                        13440 non-null  float64
 10  day                          13437 non-null  float64
 11  time                         8411 non-null   float64
 12  country                      13434 non-null  object 
 13  country.state.co

# vvv
The block below adds genus column and changes order of colums and saves the dataframe to file.
Note that the cells that are NA in the original file are now just empty cells. 

In [61]:
# create genus column: take the first word in scientific name
df['genus'] = df['scientific_name'].apply(lambda x: x.split()[0])
# rearrange to have genus sit right before scientific name
cols = df.columns.tolist()
cols = cols[:3] + [cols[-1]] + cols[3:-1]
df = df[cols]

df.to_csv('MLSN_Insects_with_genus.csv', index=False)

# vvv
Counting stuff

In [ ]:
# counts data points grouped by scientific name
name_counts = df.groupby('scientific_name').cat_num.count()
name_counts.sort_values()

In [58]:
# recount number of data points grouped by genus
genus_counts = df.groupby('genus').cat_num.count()
genus_counts.sort_values()

genus
Neocicada             1
Eriolus               1
Ephippitytha          1
Eneoptera             1
Dissosteira           1
                   ... 
Anaxipha            613
Neoconocephalus     661
Cycloptilum         736
Oecanthus          1206
Gryllus            4772
Name: cat_num, Length: 109, dtype: int64

In [59]:
# count rare occurance, genus
counts = 0
for i,v in genus_counts.items():
    if v < 10:
        print(i, v)
        counts += 1
print(counts)

Acheta 5
Acrididae 5
Antillicharis 4
Caribophyllum 4
Ceraia 9
Cocconotus 3
Diatrypa 3
Diceroprocta 1
Dissosteira 1
Eneoptera 1
Ephippitytha 1
Eriolus 1
Fidicina 2
Froggattina 8
Gryllacrididae 1
Henicopsaltria 4
Hispanogryllus 3
Idiostatus 7
Ischyra 6
Megatibicen 1
Miogryllus 2
Nemobius 1
Neocicada 1
Oreopedes 4
Paracyrtophyllus 4
Petaloptera 1
Phoebolampta 2
Phrixa 2
Plagiostira 6
Psaltoda 1
Pseudopleminia 5
Romalea 3
Scapsipedus 4
Steiroxys 3
Syntechna 1
Teleogryllus 2
Tibicen 9
Xabea 1
Xenogryllus 9
39
